In [2]:
import os
import uuid
import torch
import requests
from PIL import Image
from pymongo import MongoClient
from torchvision import models, transforms

# --- Load pretrained ResNet18 ---

In [3]:
model = models.resnet18(pretrained=True)
model.eval()

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Lenovo/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:45<00:00, 1.04MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# --- Load ImageNet labels ---

In [4]:
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_labels = requests.get(LABELS_URL).text.strip().split("\n")

# --- Transform for ResNet input ---

In [5]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# --- MongoDB Setup ---

In [6]:
client = MongoClient("mongodb://localhost:27017")
db = client["image_gallery"]
collection = db["images"]

# --- Folder containing images ---

In [7]:
FOLDER_PATH = "downloaded_images"

# --- Classify and store metadata ---

In [8]:
def classify_and_store(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        img_t = transform(image).unsqueeze(0)
        with torch.no_grad():
            out = model(img_t)
        _, index = torch.max(out, 1)
        category = imagenet_labels[index[0]]

        # Generate a unique UUID
        unique_id = str(uuid.uuid4())

        # Check if UUID already exists (rare, but safe)
        while collection.find_one({"_id": unique_id}):
            unique_id = str(uuid.uuid4())

        # Insert into MongoDB
        doc = {
            "_id": unique_id,
            "file_path": image_path.replace("\\", "/"),
            "category": category
        }
        collection.insert_one(doc)
        print(f"Inserted: {doc}")
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# --- Run over folder ---

In [9]:
for filename in os.listdir(FOLDER_PATH):
    full_path = os.path.join(FOLDER_PATH, filename)
    if os.path.isfile(full_path) and filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        classify_and_store(full_path)

Inserted: {'_id': '0b34e350-bcab-45ce-80bf-962b92a71d19', 'file_path': 'downloaded_images/-3LtGq_RPcY.jpg', 'category': 'alp'}
Inserted: {'_id': '2f61ff6d-17e7-4495-b5fc-937c5879eee6', 'file_path': 'downloaded_images/-6zl660pz3U.jpg', 'category': 'sports car'}
Inserted: {'_id': '4ff739d4-e6d4-47cc-91b8-3a6a32091bed', 'file_path': 'downloaded_images/-81lVsfM4gQ.jpg', 'category': 'Egyptian cat'}
Inserted: {'_id': 'e24dbe7d-7c22-479e-a5fb-983a50a1c034', 'file_path': 'downloaded_images/-84767CQrS8.jpg', 'category': 'shower curtain'}
Inserted: {'_id': '282740e0-2d6f-4f6d-b39d-4ece0b754ca0', 'file_path': 'downloaded_images/-aAucsS1f24.jpg', 'category': 'cabbage butterfly'}
Inserted: {'_id': 'ee5770ca-4f78-4a00-b29b-d4579b8359b5', 'file_path': 'downloaded_images/-Bq3TeSBRdE.jpg', 'category': 'church'}
Inserted: {'_id': '9aa00d95-ca59-4d32-ba3a-1fa1fe12eecd', 'file_path': 'downloaded_images/-EXF9shcTO0.jpg', 'category': 'racer'}
Inserted: {'_id': 'a3cace04-8072-4e71-bb08-607815de5739', 'file_p